第一种方法是结构化的方法。首先思考结构化的方法。如果信息整理难度过大就走统计学的路。

第二种方法是统计学的方法。

第三种方法是手动导入。至少对于我个人看过的番剧来说，手动导入是可以实现的。另外，如果有社区的话，实现起来也非常容易。但是，这就不能体现技术上的geek感。




注：如果采用selenium之类的，可以使用这个

```js
$('.mic-evaluate')[0].querySelector('span').innerText

// or 
document.querySelector('.mic-evaluate').querySelector('span').innerText


`御坂美琴：佐藤利奈
佐天泪子：伊藤加奈惠
初春饰利：丰崎爱生
白井黑子：新井里美
麦野沈利：小清水亚美
绢旗最爱：赤﨑千夏
一方通行：岡本信彦
芙兰达·赛维伦：内田真礼
布束砥信：葉山いくみ
`
```

网址的问题

https://www.bilibili.com/bangumi/media/md427

关键是找到md后面的数字的列表，之后就好办了。

不管怎样，先验证爬虫吧。

In [11]:
import requests
from bs4 import BeautifulSoup
import re
import json
H = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0"}

In [51]:


res = requests.get('https://www.bilibili.com/bangumi/media/md427', headers=H)

In [13]:
with open("a.html", 'w', encoding='utf8') as f:
    f.write(res.text)

In [27]:
# 方法1 "actors": "..."

pattern = r'"actors"[^"]*"(.+?)"'
re.findall(pattern, res.text)[0]

'御坂美琴：佐藤利奈\\n佐天泪子：伊藤加奈惠\\n初春饰利：丰崎爱生\\n白井黑子：新井里美\\n麦野沈利：小清水亚美\\n绢旗最爱：赤﨑千夏\\n一方通行：岡本信彦\\n芙兰达·赛维伦：内田真礼\\n布束砥信：葉山いくみ\\n'

接下来就是找url的环节。



In [28]:
res = requests.get('https://www.bilibili.com/bangumi/play/ss3398', headers=H)

In [30]:
with open('b.html','w',encoding='utf8') as f:
    f.write(res.text)

In [32]:
s = BeautifulSoup(res.text)

In [47]:
atag = s.select_one(".media-title")

print("https:"+atag.attrs['href'])
print(atag.text)

https://www.bilibili.com/bangumi/media/md3398/
冰菓


# 正片开始

In [58]:
# 获取全部番剧列表，按照排行榜排序

def compose_url(page, page_size):
    tmpl = "https://api.bilibili.com/pgc/season/index/result?season_version=-1&area=-1&is_finish=-1&copyright=-1&season_status=-1&season_month=-1&year=-1&style_id=-1&order=3&st=1&sort=0&page={0}&season_type=1&pagesize={1}&type=1"
    return tmpl.format(page, page_size)

compose_url(10,100)

'https://api.bilibili.com/pgc/season/index/result?season_version=-1&area=-1&is_finish=-1&copyright=-1&season_status=-1&season_month=-1&year=-1&style_id=-1&order=3&st=1&sort=0&page=10&season_type=1&pagesize=100&type=1'

In [59]:
huge_url = compose_url(1, 10000)

In [60]:
res = requests.get(huge_url, headers=H)

In [4]:
with open('data/1-all-anime-list.json', 'w', encoding='utf8') as f:
    f.write(res.text)

NameError: name 'res' is not defined

In [5]:
obj = json.load(open('data/1-all-anime-list.json', 'r', encoding='utf8'))

In [6]:
obj['code']

0

In [7]:
obj['data']['list'].__len__()

3357

In [8]:
anime_urls = [p['link'] for p in obj['data']['list']]

In [9]:
anime_urls[:10]

['https://www.bilibili.com/bangumi/play/ss26801',
 'https://www.bilibili.com/bangumi/play/ss34430',
 'https://www.bilibili.com/bangumi/play/ss36174',
 'https://www.bilibili.com/bangumi/play/ss36429',
 'https://www.bilibili.com/bangumi/play/ss38221',
 'https://www.bilibili.com/bangumi/play/ss36170',
 'https://www.bilibili.com/bangumi/play/ss34004',
 'https://www.bilibili.com/bangumi/play/ss24588',
 'https://www.bilibili.com/bangumi/play/ss33802',
 'https://www.bilibili.com/bangumi/play/ss32982']

In [12]:
# get title and cv actor list of 1 sample

sample_url = anime_urls[0]
res = requests.get(sample_url, headers=H)
s = BeautifulSoup(res.text)

atag = s.select_one(".media-title")

print("https:"+atag.attrs['href'])
print(atag.text)

https://www.bilibili.com/bangumi/media/md22718131/
鬼灭之刃


In [13]:
import hashlib
def sha256(s):  
    h = hashlib.sha256()   
    h.update(s.encode('utf8'))   
    b = h.hexdigest().lower() 
    return b

In [14]:
# download all webpages

# download 1 webpage

def compose_webpage_store(url):
    return f'data/2-all/{sha256(url)}.html'

import os

def down_url(url):
    file = compose_webpage_store(url)
    
    if os.path.exists(file):
        return
    
    res = requests.get(url, headers=H)
    if not res.ok:
        print("ERROR", url)
        return

    with open(file, 'w', encoding='utf8') as f:
        f.write(res.text)

In [15]:
MAX_AMIME = 2500 #500

for url in anime_urls[:MAX_AMIME]:
    
    down_url(url)

ERROR https://www.bilibili.com/bangumi/play/ss5970
ERROR https://www.bilibili.com/bangumi/play/ss25727
ERROR https://www.bilibili.com/bangumi/play/ss23863
ERROR https://www.bilibili.com/bangumi/play/ss25959
ERROR https://www.bilibili.com/bangumi/play/ss4347
ERROR https://www.bilibili.com/bangumi/play/ss2628
ERROR https://www.bilibili.com/bangumi/play/ss23860


In [148]:
# foreach file, get title and list
animes = []

for url in anime_urls[:MAX_AMIME]:
    file = compose_webpage_store(url)
    
    try:
        with open(file, 'r', encoding='utf8') as f:
            text = f.read()

        s = BeautifulSoup(text)

        atag = s.select_one(".media-title")
        
        view_url = url
        url = "https:"+atag.attrs['href']
        title = atag.text
        
        animes.append({"title":title, "url": url, "viewUrl":view_url })
    except Exception as e:
        print(e)
        

[Errno 2] No such file or directory: 'data/2-all/a4996134f5334a41d48c0d73c500e26d6d4f9a25ff7e2dd46fb6a00f31b15f43.html'


In [149]:
animes[:3]

[{'title': '鬼灭之刃',
  'url': 'https://www.bilibili.com/bangumi/media/md22718131/',
  'viewUrl': 'https://www.bilibili.com/bangumi/play/ss26801'},
 {'title': '咒术回战',
  'url': 'https://www.bilibili.com/bangumi/media/md28229899/',
  'viewUrl': 'https://www.bilibili.com/bangumi/play/ss34430'},
 {'title': '工作细胞 第二季',
  'url': 'https://www.bilibili.com/bangumi/media/md28231816/',
  'viewUrl': 'https://www.bilibili.com/bangumi/play/ss36174'}]

In [150]:
# download all urls in animes
for x in animes:
    down_url(x['url'])

In [146]:
def get_list(raw):
    r = []
    char_cv_list = raw.split("\\n")
    for line in char_cv_list:
        try:
            parts = line.replace(":","：").split("：")
            assert len(parts) == 2, line
            r.append(parts)
        except Exception as e:
            print("WARN", e)
    
    return r
    
get_list(cv_list)

[['阿諾斯·波魯迪戈烏多', '鈴木達央'], ['米夏·涅庫羅', '楠木燈'], ['莎夏·涅庫洛', '夏吉優子']]

In [151]:
pattern = r'"actors"[^"]*"(.+?)"'

for x in animes:
    try:
        title = x['title']
        url = x['url']

        file = compose_webpage_store(url)

        if not os.path.exists(file):
            print("file NOT exist for url:", url)

        with open(file, 'r', encoding='utf8') as f:
            text = f.read()

        cv_list_raw = re.findall(pattern, text)[0]
        actors = get_list(cv_list_raw)
        
        x['cv'] = actors
    except Exception as e:
        print("ERROR", e, title ,url)
    
    

WARN 石川由依
WARN 茅原实里
WARN 远藤绫
WARN 寿美菜子
WARN 子安武人
WARN 户松遥
WARN 内山昂辉
WARN 悠木碧
WARN Mario..
WARN 
WARN 
WARN 筑..
WARN 
WARN 
WARN 
WARN 
WARN 松冈祯丞
WARN 茅野爱衣
WARN 日笠阳子
WARN 田村由加莉
WARN 能登麻美子
WARN 井口裕香
WARN 泽城美雪
WARN 大川透
WARN 兴津和幸
WARN Lynn
WARN 诹访部顺一
WARN 堀江由衣
WARN 森奈奈子
WARN 竹内良太
WARN 大地叶
WARN 石上静香
WARN 滨野大辉
WARN 手冢弘道
WARN 钉宫理惠
WARN 
WARN 
WARN 
WARN 
WARN 柊..
WARN 醍醐虎汰朗
WARN 森七菜
WARN 小栗旬
WARN 本田翼
WARN 倍赏千惠子
WARN 吉柳咲良
WARN 平泉成
WARN 梶裕贵
WARN 木村良平
WARN 柴田秀胜
WARN 野泽雅子
WARN 花泽香菜
WARN 佐仓绫音
WARN 
WARN 
WARN ..
WARN 
WARN 
WARN 
WARN 
WARN ",
WARN 
WARN 神木隆之介、上白石萌音、长泽雅美、市原悦子、成田凌、悠木碧、岛崎信长、石川界人、谷花音、寺杣昌纪、大原沙耶香、井上和彦、茶风林、加藤有花、花泽香菜、寺崎裕香
WARN 
WARN 
WARN ヘルヴィ・オーバ..
WARN 柊..
WARN 
WARN 入野自由 \u002F 早见沙织 \u002F 松冈茉优 \u002F 悠木碧 \u002F 小野贤章 \u002F 金子有希 \u002F 石川由依 \u002F 潘惠美
WARN 
WARN 艾涅丝妲·..
WARN 
WARN 
WARN 高杉真宙
WARN Lynn
WARN 藤井雪代
WARN 内田雄马
WARN 福岛润
WARN 田中敦子
WARN 三木真一郎
WARN 
WARN 
WARN 
WARN 
WARN 
WARN 
WARN ",
WARN 源义..
WARN 
WARN 水田山葵
WARN 大原惠美
WARN 嘉数由美
WARN 关智一
WARN 木村昴
WARN 远藤绫
WARN 钉宫理惠
WARN 渡边直美

In [152]:
animes[:10]

[{'title': '鬼灭之刃',
  'url': 'https://www.bilibili.com/bangumi/media/md22718131/',
  'viewUrl': 'https://www.bilibili.com/bangumi/play/ss26801',
  'cv': [['灶门炭治郎', '花江夏树'],
   ['灶门祢豆子', '鬼头明里'],
   ['我妻善逸', '下野纮'],
   ['嘴平伊之助', '松冈祯丞'],
   ['富冈义勇', '樱井孝宏'],
   ['鳞泷左近次', '大冢芳忠'],
   ['锖兔', '梶裕贵'],
   ['真菰', '加隈亚衣'],
   ['不死川玄弥', '冈本信彦'],
   ['产屋敷耀哉', '森川智之'],
   ['产屋敷辉利哉', '悠木碧'],
   ['产屋敷雏衣', '井泽诗织'],
   ['钢铁冢萤', '浪川大辅'],
   ['鎹鸦', '山崎巧'],
   ['佛堂鬼', '绿川光'],
   ['手鬼', '子安武人']]},
 {'title': '咒术回战',
  'url': 'https://www.bilibili.com/bangumi/media/md28229899/',
  'viewUrl': 'https://www.bilibili.com/bangumi/play/ss34430',
  'cv': [['虎杖悠仁', '榎木淳弥'],
   ['伏黑惠', '内田雄马'],
   ['钉崎野蔷薇', '濑户麻沙美'],
   ['五条悟', '中村悠一'],
   ['两面宿傩', '诹访部顺一'],
   ['禅院真希', '小松未可子'],
   ['狗卷棘', '内山昂辉'],
   ['熊猫', '关智一'],
   ['七海建人', '津田健次郎'],
   ['伊地知洁高', '岩田光央'],
   ['家入硝子', '远藤绫'],
   ['夜蛾正道', '黒田崇矢'],
   ['东堂葵', '木村昴'],
   ['禅院真依', '井上麻里奈'],
   ['三轮霞', '赤﨑千夏'],
   ['乐岩寺嘉伸', '麦人'],
   ['吉野顺平', '山谷祥生'],
   ['夏油杰', '樱井

In [156]:
import json

with open('data/3-animes.json', 'w', encoding='utf8') as f:
    json.dump(animes, f, ensure_ascii=False, indent=4)